<a href="https://colab.research.google.com/github/Henrystrasser/DataScienceSnippets/blob/main/Student_confusion_kaggle_competition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [23]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

map_charting_student_math_misunderstandings_path = kagglehub.competition_download('map-charting-student-math-misunderstandings')

print('Data source import complete.')


Data source import complete.


In [24]:
map_charting_student_math_misunderstandings_path

'/root/.cache/kagglehub/competitions/map-charting-student-math-misunderstandings'

In [25]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import random
import torch

pd.set_option("display.max_colwidth", None)


# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [26]:
import pandas as pd

# Load raw data
train2 = pd.read_csv('/root/.cache/kagglehub/competitions/map-charting-student-math-misunderstandings/train.csv')
test2  = pd.read_csv('/root/.cache/kagglehub/competitions/map-charting-student-math-misunderstandings/test.csv')


N = 2000
N = N if N<len(train2) else len(train2)
print("N",N,len(train2))
random_indices = np.random.choice(train2.index, size=N, replace=False)  # no repeats
train = train2.loc[random_indices]
print("len train",len(train))
#random_indices = np.random.choice(test2.index, size=O, replace=False)  # no repeats
test = test2

print(len(train[0:5].isna()))
print(train.head())

N 2000 36696
len train 2000
5
       row_id  QuestionId  \
2799     2799       31772   
23008   23008       33471   
8085     8085       31777   
1775     1775       31772   
19108   19108       32833   

                                                                                                                                                           QuestionText  \
2799   What fraction of the shape is not shaded? Give your answer in its simplest form. [Image: A triangle split into 9 equal smaller triangles. 6 of them are shaded.]   
23008                                       A bag contains \( 24 \) yellow and green balls. \( \frac{3}{8} \) of the balls are yellow. How many of the balls are green?   
8085                      A box contains \( 120 \) counters. The counters are red or blue. \( \frac{3}{5} \) of the counters are red.\nHow many red counters are there?   
1775   What fraction of the shape is not shaded? Give your answer in its simplest form. [Image: A triangle spli

In [27]:
#train.QuestionText.value_counts()

In [28]:
question = "This is part of a regular polygon. How many sides does it have? [Image: A diagram showing an obtuse angle labelled 144 degrees]"
df_subset = train[(train["QuestionText"] == question) & (train["Category"] == "True_Correct")]
#df_subset

In [29]:
#df_subset = train[(train["QuestionText"] == question) & (train["Category"] == "False_Misconception")]
df_subset = train[train["Misconception"] == "Incomplete"]
shuffled_df = df_subset.sample(frac=1).reset_index(drop=True)
shuffled_df.head()

,row_id,QuestionId,QuestionText,MC_Answer,StudentExplanation,Category,Misconception
0,4045,31772,What fraction of the shape is not shaded? Give your answer in its simplest form. [Image: A triangle split into 9 equal smaller triangles. 6 of them are shaded.],\( \frac{3}{9} \),because 3 out of 9 are not shaded,False_Misconception,Incomplete
1,4709,31772,What fraction of the shape is not shaded? Give your answer in its simplest form. [Image: A triangle split into 9 equal smaller triangles. 6 of them are shaded.],\( \frac{3}{9} \),there are 9 triangles and three are blank.,False_Misconception,Incomplete
2,8222,31777,A box contains \( 120 \) counters. The counters are red or blue. \( \frac{3}{5} \) of the counters are red.\nHow many red counters are there?,\( 24 \),to find the answer you have to divide 120 by 5 and my answer was 24 so there are 24 red counters.,False_Misconception,Incomplete
3,4463,31772,What fraction of the shape is not shaded? Give your answer in its simplest form. [Image: A triangle split into 9 equal smaller triangles. 6 of them are shaded.],\( \frac{3}{9} \),i think this because there are 9 triangles and 3 are not shaded in,False_Misconception,Incomplete
4,4539,31772,What fraction of the shape is not shaded? Give your answer in its simplest form. [Image: A triangle split into 9 equal smaller triangles. 6 of them are shaded.],\( \frac{3}{9} \),it is 3.9 because there is 9 triangles and there is 3 not shaded in,False_Misconception,Incomplete


In [30]:
# Build label mapping
train['label'] = train['Category'].astype(str) + ':' + train['Misconception'].fillna('NaN')
#train['label'] = str(train['Category']) + ':' + train['Misconception']
print(train[0:2]['label'] )
unique_labels = train['label'].unique().tolist()
label2id = {lab: idx for idx, lab in enumerate(unique_labels)}
id2label = {idx: lab for lab, idx in label2id.items()}
train['label_id'] = train['label'].map(label2id)

2799         False_Misconception:Incomplete
23008    False_Misconception:Wrong_fraction
Name: label, dtype: object


In [31]:
def preprocess_df(df):
  df2 = df.drop(columns=["row_id","QuestionText","Category","Misconception","StudentExplanation","MC_Answer","label","QuestionId"])
  df2["question_answer"] = train.QuestionId.astype(str) +":"+ train.MC_Answer.astype(str)
  #categorials = ["question_answer"]
  dummies = pd.get_dummies(df2['question_answer'])
  df2 = pd.concat([df2, dummies], axis=1)
  df2 = df2.drop(columns=["question_answer"])
  #df2 = pd.concat([df2, encodings], axis=1)
  return df2

In [32]:
train_preprocessed = preprocess_df(train)

In [33]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# Convert the column to a list of strings
texts = train['StudentExplanation'].astype(str).tolist()

# Tokenize all texts (batch mode)
encodings = tokenizer(
    texts,
    add_special_tokens=True,
    padding='max_length',
    truncation=True,
    max_length=32,
    return_tensors='pt'
)


In [34]:
import torch
import torch.nn as nn

In [35]:
from transformers import DistilBertModel, DistilBertTokenizer

# Load DistilBERT model and tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertModel.from_pretrained("distilbert-base-uncased")
model.eval()

# Example: if you already have your texts, you can encode them here
# batch_texts = ["example sentence 1", "example sentence 2", ...]
# encodings = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt")

# If you already have `encodings` from before, you can just use them:
with torch.no_grad():
    outputs = model(
        input_ids=encodings['input_ids'],
        attention_mask=encodings['attention_mask']
    )

# outputs.last_hidden_state shape: (batch_size, sequence_length, 768)
# Take [CLS]-equivalent token (DistilBERT does not have a CLS token,
# so we use token at position 0, which is the [CLS] placeholder from tokenizer)
cls_embeddings = outputs.last_hidden_state[:, 0, :]  # shape: (batch_size, 768)

In [36]:
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader

# Suppose you have indices for validation
# indices = validation sample positions
X_combined = np.hstack([train_preprocessed, cls_embeddings])

train_mask = np.ones(X_combined.shape[0], dtype=bool)

indices = np.random.choice(X_combined.shape[0], int(0.1*X_combined.shape[0]), replace=False)
train_mask[indices] = False

X_train = X_combined[train_mask]
y_train = train_preprocessed['label_id'][train_mask]
X_val = X_combined[indices]
y_val = train_preprocessed['label_id'].iloc[indices]
X_train = X_train.astype(np.float32)
X_val = X_val.astype(np.float32)

# Convert to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.to_numpy(dtype=np.int64), dtype=torch.long)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.to_numpy(dtype=np.int64), dtype=torch.long)

# Create datasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

# Create loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [39]:
class SimpleMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        dropout_p = 0.4
        self.dropout = nn.Dropout(dropout_p)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


In [40]:
import torch.optim as optim

dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

# Dynamically set output_dim based on the number of unique labels
output_dim = len(unique_labels)
model = SimpleMLP(input_dim=X_train_tensor.shape[1], hidden_dim=64, output_dim=output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

def train_step(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    for X_batch, y_batch in loader:
        # Ensure data is on the correct device and has the correct data type
        X_batch = X_batch.to(device, dtype=torch.float32)
        y_batch = y_batch.to(device, dtype=torch.long) # Labels for CrossEntropyLoss should be long

        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

#for epoch in range(10):
#    loss = train_step(model, loader, criterion, optimizer, device)
#    print(f"Epoch {epoch+1}, Loss: {loss:.4f}")

SimpleMLP(
  (fc1): Linear(in_features=827, out_features=64, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.4, inplace=False)
  (fc2): Linear(in_features=64, out_features=47, bias=True)
)

In [44]:
num_epochs = 40
best_val_accuracy = 0
epochs_without_improvement = 0
patience = 17  # Stop if no improvement for x consecutive epochs
best_model_state = None

model.train()  # put model in training mode

for epoch in range(num_epochs):
    running_loss = 0.0

    model.train()
    loss = train_step(model, train_loader, criterion, optimizer, device)

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss}")

    # Validation step
    model.eval()  # Evaluation mode disables dropout, batchnorm updates, etc
    val_loss = 0.0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for val_inputs, val_labels in val_loader:
            val_inputs = val_inputs.to(device)
            val_labels = val_labels.to(device)

            val_outputs = model(val_inputs)

            # If binary classification:
            # For 2 output neurons: use CrossEntropyLoss and argmax
            # Or for 1 output neuron with sigmoid, use thresholding
            loss = criterion(val_outputs, val_labels)
            val_loss += loss.item()

            # Get predicted class labels
            if val_outputs.dim() == 1 or val_outputs.shape[1] == 1:
                preds = (val_outputs > 0.5).long()
            else:
                # seems to always use this one
                preds = torch.argmax(val_outputs, dim=1)
            all_preds.append(preds)
            all_labels.append(val_labels)

    avg_val_loss = val_loss / len(val_loader)
    all_preds = torch.cat(all_preds).cpu()
    all_labels = torch.cat(all_labels).cpu()

    accuracy = (all_preds == all_labels).float().mean().item()

    if accuracy > best_val_accuracy:
        best_val_accuracy = accuracy
        epochs_without_improvement = 0
        best_model_state = model.state_dict()  # Save current best model parameters
        print("Validation accuracy improved, model saved.")
    else:
        epochs_without_improvement += 1
        print(f"No improvement for {epochs_without_improvement} epoch(s).")

    # Check early stopping condition
    if epochs_without_improvement >= patience:
        print(f"Early stopping triggered at epoch {epoch+1}.")
        break


    print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {avg_val_loss:.4f}, Accuracy: {accuracy:.4f}")

Epoch 1/40, Loss: 1.5525386249809934
Validation accuracy improved, model saved.
Epoch 1/40, Validation Loss: 1.3570, Accuracy: 0.6450
Epoch 2/40, Loss: 1.440679485337776
Validation accuracy improved, model saved.
Epoch 2/40, Validation Loss: 1.2687, Accuracy: 0.6950
Epoch 3/40, Loss: 1.338397645113761
No improvement for 1 epoch(s).
Epoch 3/40, Validation Loss: 1.1989, Accuracy: 0.6500
Epoch 4/40, Loss: 1.2749211955488773
Validation accuracy improved, model saved.
Epoch 4/40, Validation Loss: 1.0991, Accuracy: 0.7150
Epoch 5/40, Loss: 1.1873957891213267
No improvement for 1 epoch(s).
Epoch 5/40, Validation Loss: 1.0647, Accuracy: 0.6900
Epoch 6/40, Loss: 1.1230997966046918
Validation accuracy improved, model saved.
Epoch 6/40, Validation Loss: 0.9715, Accuracy: 0.7350
Epoch 7/40, Loss: 1.0481807286279243
No improvement for 1 epoch(s).
Epoch 7/40, Validation Loss: 0.9094, Accuracy: 0.7350
Epoch 8/40, Loss: 0.9845568966447261
Validation accuracy improved, model saved.
Epoch 8/40, Validati